In [45]:
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame

def testknn(k, train, test):
    result = []
    a = 'Iris-setosa'
    b = 'Iris-versicolor'
    c = 'Iris-virginica'
    #对test的每一行进行遍历
    train = normalization(train)
    test = normalization(test)
    #print(train)
    for indexs in test.index:
        tag0 = 0
        tag1 = 0
        tag2 = 0
        #用来存储train和给定数据的距离和train对应的label
        disarray = []
        #与train的每一行进行距离计算
        for trainindex in train.index:
            disarray.append([distance2(test.loc[indexs], train.loc[trainindex]),train.loc[trainindex].values[4]])
            #print("aaa",test.loc[indexs])
            #print("bbb",train.loc[trainindex])
        #对testitem和train中的数据的距离进行排序
        disarray.sort(key = takeFirst)
        #print(disarray)
        #先不给权重
        for i in range(k):
            if disarray[i][1] == a:
                tag0 += 1
            elif disarray[i][1] == b:
                tag1 += 1
            else :
                tag2 += 1
        if tag0 >= tag1 and tag0 >= tag2:
            result.append([test.loc[indexs].values[0],test.loc[indexs].values[1],test.loc[indexs].values[2],test.loc[indexs].values[3],a])
        elif tag1 >= tag2 and tag1 >= tag0:
            result.append([test.loc[indexs].values[0],test.loc[indexs].values[1],test.loc[indexs].values[2],test.loc[indexs].values[3],b])
        else:
            result.append([test.loc[indexs].values[0],test.loc[indexs].values[1],test.loc[indexs].values[2],test.loc[indexs].values[3],c])
    #print(result)
    result = DataFrame(result)
    return result


#曼哈顿距离
def distance1(testitem, trainitem):
    distance = 0
    for f in range(4):
        distance += abs(testitem.values[f] - trainitem.values[f])
    return distance

#欧氏距离
def distance2(testitem, trainitem):
    distance = 0
    for f in range(4):
        distance += (testitem.values[f] - trainitem.values[f])**2
    return distance**0.5
    

def takeFirst(element):
    return element[0]

#对数据进行正则化
def normalization(data):
    for i in range(4):
        for j in range(len(data)):
            data[[i]].values[j] = (data[[i]].values[j]-data[[i]].min()) / (data[[i]].max() - data[[i]].min())
    return data

In [56]:
#使用五折交叉验证，返回最好的k
def trainknn(train):
    result = []
    #遍历k
    for i in range(len(train)//6-1):
        kprecision = 0
        for j in range(5):
        #五折交叉验证
            train1 = train[0:int(j/5*len(train))] 
            #print("lenpart1:",len(train1))
            train2 = train[int((j+1)/5*len(train)):len(train)]
            #print("lenpart2:",len(train2))
            #拼接train
            train1 = train1.append(train2)
            #print("aaa",train1)
            test1 = train[int(j/5*len(train)):int((j+1)/5*len(train))]
            count = 0
            returnlist = testknn(i,train1,test1)
            #print("aaa",returnlist)
            #print("bbb",test1)
            for k in range(len(test1)):
                if returnlist.iloc[k][4] == test1.iloc[k][4]:
                    count += 1
            kprecision += count/len(test1)
        print("准确率：",kprecision/5)
        result.append([kprecision,i])
    result.sort(key= takeFirst, reverse=True)
    return result[0][1]


            

train = pd.read_csv('train.txt', sep = ',', header = None)
train = train.reindex(np.random.permutation(train.index))
test = pd.read_csv('test.txt',  sep = ',', header = None)

bestk = trainknn(train)
print("The best k is :",bestk)

准确率： 0.33333333333333337
准确率： 0.9416666666666668
准确率： 0.925
准确率： 0.9583333333333334
准确率： 0.9333333333333332
准确率： 0.95
准确率： 0.9416666666666668
准确率： 0.95
准确率： 0.9416666666666668
准确率： 0.9416666666666667
准确率： 0.9416666666666668
准确率： 0.9583333333333333
准确率： 0.9333333333333333
准确率： 0.9333333333333333
准确率： 0.9333333333333333
准确率： 0.9416666666666668
准确率： 0.9333333333333333
准确率： 0.9416666666666668
准确率： 0.9333333333333333
The best k is : 3


In [54]:
result = DataFrame(testknn(bestk,train,test))
result.to_csv("test.txt",header=0, index = 0)